Necessary imports

In [ ]:
!pip install -qU \
  pinecone-client==3.1.0 \
  pinecone-datasets==0.7.0 \
  sentence-transformers==2.2.2 \
  pinecone-notebooks==0.1.1

In [ ]:

import os
import re
import json
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone


api_key = '5a9fb6d6-b376-484c-b0b3-c3fbc6d21208'
pc = Pinecone(api_key=api_key)

from pinecone import ServerlessSpec

cloud = 'aws'
region = 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
index_name = 'ecom-vectordb'

import time

if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1024,
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

helper functions:


In [ ]:
#load dataset
with open('/kaggle/input/amazon-review-data-subset/amazon_review_data_subset.json', 'r') as f:
    dataset = json.load(f)

In [ ]:
dataset[0].keys()

In [ ]:
def batch_lim(n, batch_size):
    
    i_end = [0]+ [min(i + batch_size, n) for i in range(0, n, batch_size)]
    return i_end

def extract_text(product_info):
    title = product_info.get('title', '').strip()
    features = ' '.join([feature.strip() for feature in product_info.get('features', [])])
    description = product_info.get('description', '').strip()
    relevant_text = f"{title}. {features}"

    # Optionally, you could include the description if it's substantial
    if description and description.lower() != 'description':
        relevant_text += f" {description}"
    return relevant_text

def process(text):
    text = re.sub(r'[^\w\s.,]', '', text)  # Keep only words, spaces, periods, and commas
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.lower()
    return text

def create_embedding(model, text):
    embedding = model.encode(text,normalize_embeddings=True).tolist()
    return embedding

# def create_mm_embedding(model, image_url, text):
#     response = requests.get(image_url)
#     image = BytesIO(response.content)
#     embedding = torch.nn.functional.normalize(model.encode(image, text))
#     return embedding.tolist()

In [ ]:
#variables
model = SentenceTransformer('BAAI/bge-m3')



In [ ]:
n = len(dataset)
batch_size = 128
i_end = batch_lim(n, batch_size)

for i in range(len(i_end) - 1):  # Iterate up to len(i_end) - 1 to avoid IndexError
    start_idx = i_end[i]
    end_idx = i_end[i+1]
    
    ids = [dataset[j]['product_id'] for j in range(start_idx, end_idx)]
    embedding = [create_embedding(model, process(extract_text(dataset[j]))) for j in range(start_idx, end_idx)]
    meta_data = [{'title': dataset[j]['title'], 
                  'price': dataset[j]['price'], 
                  'image_url':dataset[j]['image_url'],
                  'product_features': dataset[j]['description']} for j in range(start_idx, end_idx)]
    
    record_batch = zip(ids, embedding, meta_data)
    index.upsert(vectors=record_batch)
    print(f"Batch range: ({start_idx}, {end_idx})")

In [ ]:
index.describe_index_stats()

In [ ]:
import uuid
def generate_id(img_url):
    return str(uuid.uuid5(uuid.NAMESPACE_URL, img_url))

In [ ]:
dataset = products

In [ ]:

n = len(dataset)
print(n)
i_end = batch_lim(n, batch_size)

for i in range(len(i_end) - 1):  # Iterate up to len(i_end) - 1 to avoid IndexError
    start_idx = i_end[i]
    end_idx = i_end[i+1]
    
    ids = [dataset[j]['product_id'] for j in range(start_idx, end_idx)]
    embedding = [create_embedding(model, process(extract_text(dataset[j]))) for j in range(start_idx, end_idx)]
    meta_data = [{'title': dataset[j]['title'], 
                  'price': dataset[j]['price'], 
                  'image_url':dataset[j]['image_url'],
                  'product_features': dataset[j]['description']} for j in range(start_idx, end_idx)]
    
    record_batch = zip(ids, embedding, meta_data)
    index.upsert(vectors=record_batch)
    print(f"Batch range: ({start_idx}, {end_idx})")

In [ ]:
for item in products:
    display_image_from_url(item['image_url'])

In [ ]:
index.describe_index_stats()

In [ ]:
query = "high quality gaming headphones"
xq = model.encode(query, normalize_embeddings = True).tolist()

# now query
xc = index.query(vector=xq, top_k=10, include_metadata=True)
xc

Multimodal embeddings

In [ ]:
!git clone https://github.com/FlagOpen/FlagEmbedding.git
!cd FlagEmbedding


Some manual data mining

In [ ]:
products= [{
    'product_id': generate_id("https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcQnbf8yg6b0y1YHLC_qsC0BnUU2cZNEX5iEttDos-f__uOhaDVS-rzKZqGstD3XEQbBV9h-z46ye6rsdSUYR_Hamt2T_XPb3fpAR6qHURia"),
    'main_category': 'Shoes',
    'title': 'Stylish Nike Zoom Fly 5 Running Shoes for Men - 2023 Edition',
    'price': 100,
    'image_url': 'https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcQnbf8yg6b0y1YHLC_qsC0BnUU2cZNEX5iEttDos-f__uOhaDVS-rzKZqGstD3XEQbBV9h-z46ye6rsdSUYR_Hamt2T_XPb3fpAR6qHURia',
    'features': [
        "Men's", 
        "Casual wear", 
        "Active wear", 
        "Breathable design", 
        "Durable traction",
        "Stay cool and fresh with a breathable mesh upper that enhances airflow, preventing discomfort and perspiration ",
        "during extended wear. Navigate various terrains confidently with a durable rubber outsole, providing superior ",
        "grip and traction for stability and safety. Step into style and comfort with our exclusive collection of men's ",
        "shoes, crafted to perfection. Each pair offers the perfect blend of durability, functionality, and aesthetics, ",
        "making them ideal whether you’re heading to the office, hitting the gym, or going out for a casual evening."
    ],
    'description': (
        "Stay cool and fresh with a breathable mesh upper that enhances airflow, preventing discomfort and perspiration "
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcQTwrvOgYYhdiRe95hKWurE0FLCBey7kfvRSGXeo5CM9eKYFooEkDzbDwme4NZMheNzNShofx3ajlkfWXWfMtCo3VnFW1Ry3mw_i5qN3IeXbNxXVMJSqn4lkw"),
    'main_category': 'Shoes',
    'title': 'Nike Men Air Max Alpha Trainer 5 Textured Training Sports Shoes (Size 10)',
    'price': 30,
    'image_url': "https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcQTwrvOgYYhdiRe95hKWurE0FLCBey7kfvRSGXeo5CM9eKYFooEkDzbDwme4NZMheNzNShofx3ajlkfWXWfMtCo3VnFW1Ry3mw_i5qN3IeXbNxXVMJSqn4lkw",  # Image URL is not provided
    'features': [
        "Men's", 
        "Size 10", 
        "Trainer", 
        "Lace-up Boots", 
        "Casual wear", 
        "PU upper", 
        "Cushioned footbed", 
        "PU outsole"
    ],
    'description': (
        "A pair of black textured sneakers with logo printed details and regular styling, featuring a round toe and lace-up "
        "closure. The PU upper provides durability, while the cushioned footbed ensures comfort during intense workouts. "
        "The PU outsole offers stability and grip, making these shoes ideal for tough workouts. "
        "Finish your last rep with power in the Nike Air Max Alpha Trainer 5. Max Air cushioning provides comfortable stability "
        "for lifting, whether it's a light or heavy day. The wide, flat base enhances stability and grip, allowing you to move "
        "seamlessly from station to station without sacrificing style."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcT6CsCoqFBs4lgxg2vvX2X9YaixBG6AL2Ae89QiAcl9lLqnIuLSagLvjbKSFnKiOSh42ILj0yfgif-Kh5vDNURngQYKJ71jpQ"),
    'main_category': 'Shoes',
    'title': 'Premium Adidas Adizero Adios Pro 3 Shoes for Men (SFS69)',
    'price': 12.00, 
    'image_url': 'https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcT6CsCoqFBs4lgxg2vvX2X9YaixBG6AL2Ae89QiAcl9lLqnIuLSagLvjbKSFnKiOSh42ILj0yfgif-Kh5vDNURngQYKJ71jpQ',
    'features': [
        "Suitable for all seasons: Spring, summer, autumn, winter",
        "Durable rubber outsole",
        "Superior grip and traction",
        "Designed for various terrains"
    ],
    'description': (
        "The Premium Adidas Adizero Adios Pro 3 Shoes for Men are designed for performance across all seasons, including spring, "
        "summer, autumn, and winter. These shoes feature a durable rubber outsole that provides superior grip and traction, ensuring "
        "stability and safety as you navigate various terrains with confidence."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcQrbrOwvm7h8rHEsTqKlijcbGcCC1VWhgIvyn6a1UUNLfIt5392wDDamUjVBCfRIE51iKb8G9lzZXWj4yjZ-nTc4EmUmPbErYQc33w8D2a2"),
    'main_category': 'Shoes',
    'title': 'ASIAN Men Colourblocked Sneakers (Size 8)',
    'price': 25, 
    'image_url': 'https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcQrbrOwvm7h8rHEsTqKlijcbGcCC1VWhgIvyn6a1UUNLfIt5392wDDamUjVBCfRIE51iKb8G9lzZXWj4yjZ-nTc4EmUmPbErYQc33w8D2a2',
    'features': [
        "Size 8", 
        "Round toe design", 
        "Beige and white colorblocked styling", 
        "Lace-ups detail", 
        "Synthetic upper", 
        "Height increasing non-marking PVC outsole", 
        "Lightweight compound", 
        "Orthopedic memory foam", 
        "Breathable and washable material", 
        "Suitable for all seasons", 
        "Multipurpose use: gym, running, walking, basketball, badminton, trekking, dancing, party wear", 
        "Non-slip and shockproof", 
        "Cushioned footbed", 
        "Textured and patterned outsole"
    ],
    'description': (
        "These ASIAN Men Colourblocked Sneakers in beige and white, with regular styling, feature a round toe and lace-up design. "
        "The synthetic upper is breathable, lightweight, and easily washable, making it perfect for all seasons. "
        "The height-increasing non-marking PVC outsole is made with a lightweight compound and includes orthopedic memory foam, "
        "providing extra comfort and a perfect grip. Designed for multiple uses, these sneakers are ideal for gym workouts, "
        "outdoor running, walking, basketball, badminton, trekking, dancing, and even party wear. "
        "The cushioned footbed and non-slip, shockproof outsole ensure a comfortable and secure fit with every step."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcTAax05sgNj9NMdDqiME5Ge95MKS1iIsAFSErDj22WRYPS3L9gWSMKUHKdAxgNd-7GcsWS_y25LQVe6OyXVyB0ae0lKTu-u819N3IPQlKbJ"),
    'main_category': 'Shoes',
    'title': 'Men White and Blue Memory Foam Technology Running Shoes',
    'price': 13, 
    'image_url': 'https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcTAax05sgNj9NMdDqiME5Ge95MKS1iIsAFSErDj22WRYPS3L9gWSMKUHKdAxgNd-7GcsWS_y25LQVe6OyXVyB0ae0lKTu-u819N3IPQlKbJ',
    'features': [
        "Gym shoes for men", 
        "HRX brand", 
        "Memory foam technology", 
        "Optimal support and durability", 
        "Sleek design", 
        "White and blue color scheme", 
        "Suitable for running, gym workouts, and outdoor activities", 
        "Transitions easily from day to night"
    ],
    'description': (
        "The HRX Men White and Blue Running Shoes redefine comfort and style, offering the perfect blend of functionality and fashion. "
        "Engineered with cutting-edge memory foam technology and premium materials, each pair ensures optimal support and durability, "
        "whether you're hitting the gym or exploring the outdoors. From running errands to running marathons, HRX shoes keep you "
        "stepping with confidence and ease. The sleek design and attention to detail make these shoes a standout addition to any "
        "outfit, effortlessly transitioning from day to night."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcSqynz64XseSX2a55AgxF3N-JO2gOFxo7gVgRhFhP6WcT23VLu39OS7XCI5Vc4a5fSHYjj0Oq6TxPNb2lZmMqROI_MPyLdLZdjW-Dui-ejNRnL9oVbbEjD28Wo"),
    'main_category': 'Shoes',
    'title': 'POWER Grey Textile Sports Shoes for Men (Size 9)',
    'price': 30,  
    'image_url': "https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcSqynz64XseSX2a55AgxF3N-JO2gOFxo7gVgRhFhP6WcT23VLu39OS7XCI5Vc4a5fSHYjj0Oq6TxPNb2lZmMqROI_MPyLdLZdjW-Dui-ejNRnL9oVbbEjD28Wo",  # Image URL is not provided
    'features': [
        "Size 9",
        "Running shoes",
        "Lace-up boots",
        "Casual wear",
        "Shaded grey color",
        "Breathable flyknit upper",
        "Versatile design",
        "Suitable for track, gym, and running errands"
    ],
    'description': (
        "Step into action with these shaded grey running and sports shoes from Power. Designed for versatility, these shoes feature "
        "a breathable flyknit upper and a secure lace-up closure, providing the support and style you need for any activity. Whether "
        "you're hitting the track, the gym, or simply running errands, these gym shoes for men are built to go the extra mile, ensuring "
        "comfort and performance throughout the day."
    )
},
    {
    'product_id': generate_id("https://cdn-images.cure.fit/www-curefit-com/image/upload/c_fill,q_auto:eco,dpr_2,f_auto,fl_progressive//cultgear-content/5imfVWn9ZuUxLoForJqtiqQZ"),
    'main_category': 'Shoes',
    'title': 'Women AMALGO Versatile Running Shoes',
    'price': 20,  
    'image_url': 'https://cdn-images.cure.fit/www-curefit-com/image/upload/c_fill,q_auto:eco,dpr_2,f_auto,fl_progressive//cultgear-content/5imfVWn9ZuUxLoForJqtiqQZ',
    'features': [
        "Versatile running shoes", 
        "Regular fit", 
        "Lace closure", 
        "Knit textile upper", 
        "Cushioned midsole", 
        "Heel for added stability", 
        "Soft forefoot for flexibility", 
        "Suitable for strength training and gym workouts"
    ],
    'description': (
        "Tackle your strength training and gym workouts without hesitation with the Women AMALGO Versatile Running Shoes. "
        "These shoes are designed to help you level up your game, featuring a regular fit with a lace closure and a knit textile upper "
        "for a comfortable and secure feel. The cushioned midsole keeps your feet comfortable, while the heel adds stability for lifting. "
        "A soft forefoot provides the flexibility you need during agility and mobility exercises, making these shoes the perfect choice "
        "for both style and performance."
    )
},
    {
    'product_id': generate_id("https://tego.fit/cdn/shop/products/2_a543f5bb-d826-4682-8ce8-b4bd21cc0346_1800x1800.jpg?v=1677663261"),
    'main_category': 'Fitness Equipment',
    'title': 'STANCE Reversible Yoga Mat with GuideAlign™ and Non-Slip Grip',
    'price': 20,
    'image_url': 'https://tego.fit/cdn/shop/products/2_a543f5bb-d826-4682-8ce8-b4bd21cc0346_1800x1800.jpg?v=1677663261',
    'features': [
        "Reversible yoga mat",
        "GuideAlign™ for posture alignment",
        "Honeycomb texture for non-slip grip",
        "Stability enhancing design",
        "Suitable for all types of workouts"
    ],
    'description': (
        "The STANCE Yoga Mat is a truly reversible mat with a functional design to add stability to your workout. One side features "
        "GuideAlign™ to help keep your posture in check, allowing for better balance, deeper stretches, and the full benefit of each pose. "
        "On the reverse side, the honeycomb texture provides a confident non-slip grip, perfect for extended sweaty sessions. "
        "No slipping, no shaking—just a perfect stance every time."
    )
}, 
    {
    'product_id': generate_id("https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcSwFYUC0HJbZopMM0qtjFeKQoIupIhanF5i958oOQeCeABB07r7MAvAclJqguWt6k19Avnz_G4E3qHyIKYDkEbh_BpZ2VVbhN06YFcOm_Q0Fv8hefmn5lqdfw&usqp=CAE"),
    'main_category': 'Fitness Equipment',
    'title': 'HACER Unisex Anti-Slip Yoga Kit with Carry Bag, Gloves & Socks',
    'price': 10,
    'image_url': 'https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcSwFYUC0HJbZopMM0qtjFeKQoIupIhanF5i958oOQeCeABB07r7MAvAclJqguWt6k19Avnz_G4E3qHyIKYDkEbh_BpZ2VVbhN06YFcOm_Q0Fv8hefmn5lqdfw&usqp=CAE',
    'features': [
        "Unisex yoga kit",
        "Anti-slip cotton yoga mat",
        "Includes carry bag, gloves, and socks",
        "Made from eco-friendly materials",
        "Dual-sided texture for traction",
        "Anti-tear durability",
        "Easy to clean"
    ],
    'description': (
        "The HACER Unisex Anti-Slip Yoga Kit is perfect for both men and women looking to enhance their fitness routine. "
        "This comprehensive kit includes a premium quality anti-skid cotton yoga mat, carry bag, gloves, and socks. "
        "Made from eco-friendly materials and featuring a dual-sided texture for excellent traction, this mat offers good elasticity "
        "and anti-tear durability, making it easy to clean and ideal for regular use."
    )
},
    {
    'product_id': generate_id("https://demo.artshoppy.com/media/catalog/product/cache/ec4b9a237d98decac940874839d2132d/l/u/luma-yoga-kit-2.jpg"),
    'main_category': 'Fitness Equipment',
    'title': 'Unisex Yoga Kit with PVC Non-Slip Surface and Thera-Band Exercise Ball',
    'price': 9,
    'image_url': 'https://demo.artshoppy.com/media/catalog/product/cache/ec4b9a237d98decac940874839d2132d/l/u/luma-yoga-kit-2.jpg',
    'features': [
        "Unisex yoga kit",
        "PVC non-slip surface",
        "Includes Thera-Band Standard Exercise Ball",
        "Comes with two plugs and an inflation adapter",
        "Thera-Band Exercise Ball Guide Poster with 24 exercises",
        "Available in five sizes for height variations"
    ],
    'description': (
        "This Unisex Yoga Kit is designed for versatility and safety, featuring a PVC non-slip surface to ensure stability during workouts. "
        "It includes a Thera-Band Standard Exercise Ball, ideal for strengthening muscles, improving posture, and preventing back pain. "
        "The kit comes with two plugs, an inflation adapter, and a detailed Exercise Ball Guide Poster showcasing 24 exercises. "
        "Available in five sizes, this kit caters to users of various heights, making it an excellent choice for a comprehensive fitness routine."
    )
},
    {
    'product_id': generate_id("https://www.atharvanlife.com/wp-content/uploads/2022/05/eco-friendly-yoga-accessories-advanced-kit-1-800x800.jpg"),
    'main_category': 'Fitness Equipment',
    'title': 'Eco-friendly Yoga Accessories Advanced Kit',
    'price': 12,
    'image_url': 'https://www.atharvanlife.com/wp-content/uploads/2022/05/eco-friendly-yoga-accessories-advanced-kit-1-800x800.jpg',
    'features': [
        "Eco Corner Textured Cork Yoga Mat",
        "Eco Corner Brick",
        "Yoga Bag",
        "Mudra Coaster",
        "Complete yoga kit",
        "Eco-friendly materials"
    ],
    'description': (
        "The Eco-friendly Yoga Accessories Advanced Kit includes everything you need for a complete and sustainable yoga practice. "
        "Featuring an Eco Corner Textured Cork Yoga Mat, Eco Corner Brick, Yoga Bag, and a Mudra Coaster, this kit is designed with "
        "eco-friendly materials to enhance your yoga experience. Perfect for yogis who value sustainability without compromising on quality."
    )
},
    {
    'product_id': generate_id("https://a.media-amazon.com/images/I/31ZTh-8qOpL.jpg"),
    'main_category': 'Fitness Equipment',
    'title': 'Upgraded Eco-friendly Non-Slip Yoga Mat (Athleisure)',
    'price': 18,
    'image_url': 'https://a.media-amazon.com/images/I/31ZTh-8qOpL.jpg',
    'features': [
        "Upgraded anti-skid design",
        "Eco-friendly material (TPE)",
        "4mm thickness for optimal comfort",
        "Non-slip textured surface",
        "Suitable for all types of yoga, Pilates, and floor exercises",
        "Latest technological improvement over traditional mats"
    ],
    'description': (
        "The Upgraded Eco-friendly Non-Slip Yoga Mat is designed for optimal performance and comfort. Made with a patent-pending eco-friendly material, "
        "this mat offers superior grip with its non-slip textured surface, making it ideal for all types of yoga, Pilates, and floor exercises. "
        "With a 4mm thickness, it provides the perfect balance of cushioning and stability, ensuring a comfortable experience for yogis of all levels."
    )
},
    {
    'product_id': generate_id("https://proyog.com/cdn/shop/files/The-Essential-Yoga-Saver-Pack1_5000x.jpg?v=1695130934"),
    'main_category': 'Fitness Equipment',
    'title': 'The Essential Yoga Saver Pack',
    'price': 25,
    'image_url': 'https://proyog.com/cdn/shop/files/The-Essential-Yoga-Saver-Pack1_5000x.jpg?v=1695130934',
    'features': [
        "The Bengal Mat: Jute and Natural Rubber",
        "The Jod Mat Band",
        "The Bharat Mat Bag: Denim material",
        "The Saucha Mat Cleaner with Lavender Oil",
        "Complimentary E-book on Strength and Stability Flow with Samrat Pasham",
        "Eco-friendly and durable materials"
    ],
    'description': (
        "((Athleisure))Elevate your yoga practice with The Essential Yoga Saver Pack, a comprehensive kit curated to enhance every aspect of your yoga journey. "
        "This pack includes The Bengal Mat, crafted from Jute and Natural Rubber for eco-friendly durability, The Jod Mat Band, and The Bharat Mat Bag "
        "fashioned from Denim for convenient transportation. The Saucha Mat Cleaner with Lavender Oil ensures your mat stays fresh, and as a bonus, "
        "you'll receive a complimentary E-book on Strength and Stability Flow with Samrat Pasham."
    )
},
    {
    'product_id': generate_id("https://triprindia.com/cdn/shop/files/TOGRRN-D141_700x.jpg?v=1708091371"),
    'main_category': 'Apparel',
    'title': 'Olive Green Black Printed Gym bodybuilding T-Shirt',
    'price': 20,
    'image_url': 'https://triprindia.com/cdn/shop/files/TOGRRN-D141_700x.jpg?v=1708091371',
    'features': [
        "Black graphic print adds a playful and trendy element",
        "Olive Green color with a round neck for stylish contrast",
        "Crafted from a comfortable fabric blend",
        "Knitted cotton blend fabric (170-180 gsm)",
        "Regular fit",
        "Visually appealing and versatile ensemble",
        "Suitable for various occasions"
    ],
    'description': (
        "Fit for bodybuilding workout"
        "The Olive Green Black Printed T-Shirt combines style and functionality with its eye-catching graphic print and comfortable fabric blend. "
        "Featuring a round neck and a regular fit, this t-shirt is crafted from a knitted cotton blend fabric, ensuring both comfort and durability. "
        "The Olive Green color provides a stylish contrast, making this t-shirt a versatile choice for various occasions."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcSWO3auUYfKkIwHgGG4iSj9uuNZ9n96D_T1yNzeAt6n7cPa5xaF_-FoDk-N0n4xAxe31CPuNfMfW0WUgFsyFifVmRpAVDjwWyyyW3WXhIg&usqp=CAE"),
    'main_category': 'Apparel',
    'title': 'Technosport Solid Men Round Neck Maroon Gym T-Shirt',
    'price':15,
    'image_url': 'https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcSWO3auUYfKkIwHgGG4iSj9uuNZ9n96D_T1yNzeAt6n7cPa5xaF_-FoDk-N0n4xAxe31CPuNfMfW0WUgFsyFifVmRpAVDjwWyyyW3WXhIg&usqp=CAE',
    'features': [
        "100% Polyester fabric",
        "Cooling effect to stay dry and comfortable",
        "Moisture-wicking and quick-dry technology",
        "Crew neck style",
        "Short sleeves",
        "Solid color in Fig",
        "Fashionable yet functional design",
        "Ideal for sports and active wear"
    ],
    'description': (
        "Best for after gym rest. Comfortable for bodybuilding lifts"
        "Stay cool and comfortable on the move with the Technosport Solid Men Round Neck Maroon T-Shirt. Made from 100% Polyester, this t-shirt features "
        "a cooling effect with moisture-wicking and quick-dry technology, ensuring you stay fresh and dry during any activity. The crew neck design and "
        "short sleeves provide a classic look, while the solid Fig color adds a trendy touch. Perfect for the modern adventurer, this t-shirt combines "
        "style and functionality for any occasion."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcQX7JjxfiFIGeCWmnsLC-GkgLvxBFq6RKpcjmj1FrRlIlCkXVMhSSgunOhPAbK93GWfuaG__ak2L1iDsQUvrr1USAsuESLO2OsMJ51ZtJABjFntqHyupBnN3Es"),
    'main_category': 'Apparel',
    'title': 'Girls Striped A-line Top',
    'price': 15,
    'image_url': 'https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcQX7JjxfiFIGeCWmnsLC-GkgLvxBFq6RKpcjmj1FrRlIlCkXVMhSSgunOhPAbK93GWfuaG__ak2L1iDsQUvrr1USAsuESLO2OsMJ51ZtJABjFntqHyupBnN3Es',
    'features': [
        "Blue and yellow vertical stripes",
        "Round neck",
        "Short, regular sleeves",
        "Knitted and woven fabric",
        "A-line fit",
        "Cotton fabric"
    ],
    'description': (
        "The Girls Striped A-line Top features a playful and colorful design with blue and yellow vertical stripes. The round neck and short, regular sleeves provide a classic look, "
        "while the A-line cut adds a fun and flattering silhouette. Made from cotton and featuring a combination of knitted and woven fabrics, this top is both skin-friendly and stylish, "
        "making it a perfect gift for sisters and a cheerful addition to any young girl's wardrobe."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcSueSHjNu--deSYGJ_OiyCjBAK_hp_BmU9bgpTvlOb3_dP_M9M0JMEYSrAZSzCZFJubFmOEqvecItgCDNcnfCB4K8ig9KfrvWAw0LFNfQ"),
    'main_category': 'Apparel',
    'title': 'Stylo Bug Girls Striped Pure Cotton Regular Top',
    'price': 10,
    'image_url': 'https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcSueSHjNu--deSYGJ_OiyCjBAK_hp_BmU9bgpTvlOb3_dP_M9M0JMEYSrAZSzCZFJubFmOEqvecItgCDNcnfCB4K8ig9KfrvWAw0LFNfQ',
    'features': [
        "Navy blue and white vertical stripes",
        "Mandarin collar",
        "Long, cuffed sleeves",
        "Tie-up detail",
        "Woven cotton fabric",
        "Button closure",
        "Regular fit"
    ],
    'description': (
        "The Stylo Bug Girls Striped Pure Cotton Regular Top is a charming and stylish addition to any young girl's wardrobe. Featuring navy blue and white vertical stripes, "
        "a mandarin collar, and long, cuffed sleeves with a tie-up detail, this top combines classic design with a modern touch. Made from woven cotton and with a button closure, "
        "it's comfortable and easy to wear, making it a great gift for a younger sister or any young fashion enthusiast."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcS0CNZTcDKJRY-ZfMFtzwQMKoSzQ32DhS4idynIQsyOHQZTA3PMgC43HZmuyo4t9V-vpbpnDVGISmI_oi3KH2k5CA9HcfZ3JWaVUkPxmzF3ERUq-7NqBdbe&usqp=CAE"),
    'main_category': 'Apparel',
    'title': 'Sports/Gym T-Shirt for Men Quick Dry Fit Jersey',
    'price': 20,
    'image_url': 'https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcS0CNZTcDKJRY-ZfMFtzwQMKoSzQ32DhS4idynIQsyOHQZTA3PMgC43HZmuyo4t9V-vpbpnDVGISmI_oi3KH2k5CA9HcfZ3JWaVUkPxmzF3ERUq-7NqBdbe&usqp=CAE',
    'features': [
        "100% Polyester fabric",
        "Quick-dry and sweat-absorbing",
        "Durable with no color bleeding",
        "Premium quality fabric",
        "Manufactured using advanced technologies",
        "Soft and comfortable feel",
        "Designed for athletes"
    ],
    'description': (
        "The Sports T-Shirt for Men Quick Dry Fit Jersey is designed to enhance your performance and comfort. Made from 100% Polyester, "
        "this t-shirt features advanced moisture-wicking technology that absorbs and evaporates sweat quickly, ensuring you stay dry and comfortable. "
        "With its durability, soft feel, and premium fabric, it's perfect for athletes and active individuals looking for a reliable gym shirt."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcRBqEzpidQSodaDWGOP7fogSkUyHSIEFbGAxyiYDV_rZl1VeTe5HqWozGpPnkE3qYLHL8ycNL5l-CVuPbU4OSIJQVmiB_AKh0ak-cbHtFbaA4l-23PE1XPw"),
    'main_category': 'Apparel',
    'title': 'Stylo Bug Girls Floral Printed Cotton TShirt Style Top (11-12Y)',
    'price': 17, 
    'image_url': 'https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcRBqEzpidQSodaDWGOP7fogSkUyHSIEFbGAxyiYDV_rZl1VeTe5HqWozGpPnkE3qYLHL8ycNL5l-CVuPbU4OSIJQVmiB_AKh0ak-cbHtFbaA4l-23PE1XPw',
    'features': [
        "Green, yellow, and blue floral print",
        "Shirt collar",
        "Three-quarter puff sleeves",
        "Woven fabric",
        "Button closure",
        "Breathable fabric",
        "Non-toxic dye",
        "Regular fit"
    ],
    'description': (
        "The Stylo Bug Girls tops Floral Printed Cotton Shirt Style Top is perfect for young girls who love vibrant and playful designs. With a green, yellow, and blue floral print, "
        "a classic shirt collar, and three-quarter puff sleeves, this top combines style with comfort. The breathable, woven cotton fabric and non-toxic dye ensure both safety and ease of wear. "
        "Ideal as a gift for a growing sister or child, it is designed for a regular fit and perfect for various occasions."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcSERYHa3FZEYIvmk8563OTWTqOVuoYaXniALNMzpx5RIlINFOAjmwiRMYjjGO1pjMIfNRbUmVRO7EcFX_9t6zk3rF2jdFlZpNArW7Mtfnld"),
    'main_category': 'Apparel',
    'title': 'Girls Solid Formal Grey tops',
    'price': 20,  
    'image_url': 'https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcSERYHa3FZEYIvmk8563OTWTqOVuoYaXniALNMzpx5RIlINFOAjmwiRMYjjGO1pjMIfNRbUmVRO7EcFX_9t6zk3rF2jdFlZpNArW7Mtfnld',
    'features': [
        "Solid grey color",
        "3/4th sleeves",
        "Curved collar",
        "Applique ornamentation",
        "Regular fit",
        "Crepe fabric",
        "Hand wash care"
    ],
    'description': (
        "The Girls Solid Formal Grey Shirt is a versatile and stylish choice for formal occasions. Made from crepe fabric, this shirt features a solid grey color, a curved collar, and 3/4th sleeves. "
        "The applique ornamentation adds a touch of sophistication, while the regular fit ensures comfort. Perfect for office and meetings, this shirt is designed for western wear and is easy to maintain with hand wash care."
    )
},
    {
    'product_id': generate_id("https://cottonworld.net/cdn/shop/files/L-BLOUSE-17268-20929-CHAMBRAYBLUE_4.jpg?v=1714712765&width=960"),
    'main_category': 'Apparel',
    'title': 'Women\'s Chambray Blue Linen Loose Fit Blouse',
    'price': 25,  # Price not provided
    'image_url': 'https://cottonworld.net/cdn/shop/files/L-BLOUSE-17268-20929-CHAMBRAYBLUE_4.jpg?v=1714712765&width=960',
    'features': [
        "Chambray blue color",
        "100% linen fabric",
        "Loose fit",
        "Round V-neck",
        "Long sleeves with cuffs",
        "No pockets",
        "Solid print"
    ],
    'description': (
        "The Women's Chambray Blue Linen Loose Fit Blouse is a stylish and comfortable choice for casual wear. Made from 100% linen, this blouse features a chambray blue color and a loose fit, "
        "making it perfect for both casual outings and office wear. The round V-neck and long sleeves with cuffs add a touch of elegance, while the solid print ensures a versatile and timeless look."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcTWzCA4MoyaN-1LEGpuHxPwDxDQ28p9mjL8LkQii2oZ6uZgTCUsIMzc4IyA4nA-8LmLkYn4jvcLZRaC7sQ1FRQBQytBUtII1DdmPKLrp-08WgKH6F4k5QL-uQ"),
    'main_category': 'Apparel',
    'title': 'Basics By Tokyo Talkies Off White V Neck Fitted Tops',
    'price': 23,  # Price not provided
    'image_url': 'https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcTWzCA4MoyaN-1LEGpuHxPwDxDQ28p9mjL8LkQii2oZ6uZgTCUsIMzc4IyA4nA-8LmLkYn4jvcLZRaC7sQ1FRQBQytBUtII1DdmPKLrp-08WgKH6F4k5QL-uQ',
    'features': [
        "Off white color",
        "Solid print",
        "V-neck design",
        "Long sleeves",
        "Knitted polyester fabric",
        "Fitted style"
    ],
    'description': (
        "The Basics By Tokyo Talkies Off White V Neck Fitted Tops offer a chic and versatile addition to your wardrobe. Featuring a solid off white color and a V-neck design, "
        "this top is made from knitted polyester fabric for a comfortable fit. With long sleeves and a fitted style, it is ideal for casual wear as well as office and party settings."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcShBHegG0jAPK-RR1CdeERqiC8TH3c25y04HmH6AKnvRosgNsIjXzgZ-Nal-2zoiVFm6bWqAbe_L0UK3VGzS1o9TUFia77RDIeAcW7x7Xn7"),
    'main_category': 'Apparel',
    'title': 'Charcoal T-shirt for Women',
    'price': 20,
    'image_url': 'https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcShBHegG0jAPK-RR1CdeERqiC8TH3c25y04HmH6AKnvRosgNsIjXzgZ-Nal-2zoiVFm6bWqAbe_L0UK3VGzS1o9TUFia77RDIeAcW7x7Xn7',
    'features': [
        "Solid charcoal color",
        "Brand logo printed",
        "Regular length",
        "Round neck",
        "Sleeveless",
        "Woven polyester fabric"
    ],
    'description': (
        "This Charcoal T-shirt for women combines style and functionality with its solid color and brand logo print. Designed with a round neck and sleeveless cut, it is crafted from woven polyester fabric, "
        "providing both comfort and durability. Ideal for casual wear or workouts, this t-shirt ensures a sleek and modern look."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcSuldPMXAQdQV6BpPK8OlvT6CfoVgvPP6Hg8u98__Kv5ANZjkGhHXJhu_50dJIQRjn1gd8ZSczs600RVpemuwzX13SKrp_p&usqp=CAY"),
    'main_category': 'Apparel',
    'title': 'Yoga T-shirt for Ladies',
    'price': 20,  # Price not provided
    'image_url': 'https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcSuldPMXAQdQV6BpPK8OlvT6CfoVgvPP6Hg8u98__Kv5ANZjkGhHXJhu_50dJIQRjn1gd8ZSczs600RVpemuwzX13SKrp_p&usqp=CAY',
    'features': [
        "Soft and lightweight fabric",
        "Breathable",
        "Eco-friendly",
        "Ideal for yoga and workouts"
    ],
    'description': (
        "The Yoga T-shirt for Ladies is perfect for both lounging and workouts. Made from a soft and lightweight fabric, this t-shirt ensures maximum comfort while being eco-friendly. "
        "Its breathable material makes it suitable for yoga and other fitness activities, blending functionality with sustainability."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcQFJPAILbjShYaVt6fo4Q_IAR_CzLddMoWrP-80dHa6MqkLTdJpvPrKnNoP7G_cp62vzDnv-56Wyf1Mt_QpUscIFYRo3ei82A&usqp=CAY"),
    'main_category': 'Apparel',
    'title': 'Women\'s Barefoot Relaxed Fit Yoga T-shirt',
    'price': 30,
    'image_url': 'https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcQFJPAILbjShYaVt6fo4Q_IAR_CzLddMoWrP-80dHa6MqkLTdJpvPrKnNoP7G_cp62vzDnv-56Wyf1Mt_QpUscIFYRo3ei82A&usqp=CAY',
    'features': [
        "Relaxed fit",
        "Soft and breathable fabric",
        "Ideal for yoga",
        "Comfortable for all-day wear"
    ],
    'description': (
        "The Women's Barefoot Relaxed Fit Yoga T-shirt is designed for ultimate comfort during yoga sessions. With a relaxed fit and soft, breathable fabric, this t-shirt is perfect for both exercise and leisure. "
        "The design ensures ease of movement, making it a versatile choice for all-day wear."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcTnCvFw8gufu2TuGoy55OIXRNgJ7lft-3YELafZjlTzvPz3Sk3TibTwxPTkkbASup6tyJixxsVlqys69Ka1ylcxv5V25Pw_ywCgtmIUZFRNx2DkBUWeoRBX&usqp=CAE"),
    'main_category': 'Apparel',
    'title': 'Women’s Sport\'s Cotton T-Shirt Purple',
    'price': 12,
    'image_url': 'https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcTnCvFw8gufu2TuGoy55OIXRNgJ7lft-3YELafZjlTzvPz3Sk3TibTwxPTkkbASup6tyJixxsVlqys69Ka1ylcxv5V25Pw_ywCgtmIUZFRNx2DkBUWeoRBX&usqp=CAE',
    'features': [
        "Organic cotton fabric",
        "Purple color",
        "Regular fit",
        "Soft and breathable",
        "Supports NGOs dedicated to preserving the environment"
    ],
    'description': (
        "The Women’s Sport's Cotton T-Shirt in purple is crafted from organic cotton for a soft and comfortable feel. Designed for gentle yoga sessions, it features a regular fit and breathable fabric. "
        "This t-shirt not only offers comfort but also supports environmental sustainability with a portion of your purchase going to NGOs dedicated to preserving the planet."
    )
},
    {
    'product_id': generate_id("https://www.rareism.com/cdn/shop/files/6504258261720094768____17666_modelswapprocessed_765x.jpg?v=1720264541"),
    'main_category': 'Apparel',
    'title': 'Boxy Fit T-shirt with Graphic Print',
    'price': 30,
    'image_url': 'https://www.rareism.com/cdn/shop/files/6504258261720094768____17666_modelswapprocessed_765x.jpg?v=1720264541',
    'features': [
        "Boxy fit",
        "Graphic print",
        "Half sleeves",
        "Soft and breathable fabric",
        "Easy styling with jeans, shorts, or skirts"
    ],
    'description': (
        "The Boxy Fit T-shirt with Graphic Print offers a relaxed silhouette with an eye-catching graphic for added style. Featuring half sleeves and a soft, breathable fabric, this t-shirt is perfect for everyday wear. "
        "It pairs effortlessly with jeans, shorts, or skirts, making it a versatile addition to any wardrobe."
    )
},
    {
    'product_id': generate_id("https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcTowzH2RXtc0CFWZ34k4WIYVxlsoLChVK8w8nFNMPm9HJlJbR2DwCAAm6lIz_DZONMC47Jy-RxhcYwydxBCsDouY39bAUDcNA&usqp=CAY"),
    'main_category': 'Apparel',
    'title': 'Cotton Printed Women T-shirt Regular Fit',
    'price': 15,
    'image_url': 'https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcTowzH2RXtc0CFWZ34k4WIYVxlsoLChVK8w8nFNMPm9HJlJbR2DwCAAm6lIz_DZONMC47Jy-RxhcYwydxBCsDouY39bAUDcNA&usqp=CAY',
    'features': [
        "Blue color",
        "Minimalistic design",
        "100% cotton",
        "Regular fit",
        "Ideal for gym and yoga"
    ],
    'description': (
        "The Cotton Printed Women T-shirt in blue offers a minimalistic design with a focus on comfort. Made from 100% cotton, this t-shirt provides a relaxed fit, making it perfect for both gym workouts and yoga sessions. "
        "Its soft fabric ensures comfort throughout your activities."
    )
}

]

In [ ]:
#image-vectordb
online_imgs = [
 {'title':"steel wall hanging", 'image_url':"https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcSjqOdGZ0hU6sj0I0L48nef3tFm7U9FyFbJnEisaE5kQbEDwpNS75_HPVAAuOjp8qj2XTL2RvBfnS1z7qOW6aTJQ-FnieM5S_0idjk6sH5v&usqp=CAE"},
 {'title':"Japanese Bonsai Art Wall Art | Bonsai Art Canvas Print | Canvas Art Bay", 'image_url':"https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcSERalrChWs6cCQutu9olDQJ8V-OzjPwHgwLqo0wYMWab7YVncOcJ2PTU2lHg3JNH6cqji1ap-dfdYnySelFs4t1Jgud-dhRSFkjxuUMKoB&usqp=CAE"},
 {'title':"Love Craft Gifts Personalized Wooden Panda Lmp With Name", 'image_url':"https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcTPSLW4vHYsAo1JHeNKBL3vGhr8IZgvX0YoisJlYm1-Jf45QPWe7bQbufR9_oxY3GqCKuQOHppV0o5X3DEDchWkcsOT3dXBcY4pMsTctuBIrn49zNEKRUZGSQ&usqp=CAE"},
 {'title':"Cute Panda Light Lamp For Kids", 'image_url':"https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcTsWH_MsGc90i-YTiv4z0r287osaO5sOOC_5YaOCRliD0IQpp5yVmlOdeNRTK7CMZAxQCIsr5WXOobipIa589OkbDh3TyyIr5vDoiJ3EFitzujxcNCjl8WFaQ&usqp=CAE"},
 {'title':"Kids animal table lamp", 'image_url':"https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcRCGLgWo2dodcilKVrAIOKckupt6GGedmWKqSSaC_gvgOGCirIUKGUzPMsoKAEz6hweoCQzpkq00NHVz9b1clgk5mLwdzl1Z9dpZ9iZ5H35G0Zeqephv8HR6w&usqp=CAE"},
 {'title':"Ceramic vase blue flowers", 'image_url':"blob:https://huggingface.co/ec97414f-227f-4cfa-98f2-021e962335bc"},
 {'title':"Ceramic vase for home", 'image_url':"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTTAQUUI_rcb-WoKfAA9jSBIXv6wiRkgV3HCA&s"},
 {'title':"red flower vase", 'image_url':"blob:https://huggingface.co/d56b952c-0e55-4607-be54-01f09acd8c77"},
 {'title':"Ceramic Floral Vase", 'image_url':"blob:https://huggingface.co/a02e9c43-c04b-4a36-b066-c6e8d17fa1a9"},
 {'title':"Redwolf Marvel Official T-shirt-S Black", 'image_url':"https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcS1NgEHIL-sBFDdqeImiKTtrOhdPoCGlH3XuPuSow536jHlNwcACRJLwPGe2ySm64gSPVZjmTr3sPpBi253TD8Aj-zfyYuPFfT1j3sHuzzggzcO9vvISR8p&usqp=CAE"},
 {'title':"Cotton Blend Tshirt For Men ", 'image_url':"https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcT8f66D35W76A9di793acSXJm5sLm2vDE9HrvuSIQcbLQv7RASCE7ptqsI90cfM11txqYcla-urDmTTsemv5R0gb9RMeuGI"},
 {'title':"Bewakoof Brown Tom & Jerry Printed Round Neck Short Sleeves Cotton Oversized T-shirt (M)", 'image_url':"https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcSD21adA-P8A66SygGUtLBMKA-gje61QyE3SbZM2mlbtNWi-Nw0fvX2zN0MFyoMZ4HSpE3jSZxHqTr39FpPcA672TqETd2_No51n7DcsVI&usqp=CAE"},
 {'title':'Unisex OffWhite and Blue Rain Boots - Hopscotch | Size - 34.5', 'image_url':"https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcQ1I6HgKTOIUmiVWNbrZcYAmAGg1gABFLqfpWngGCz5mMPdW3s3d0kJt52HLVYJIGCHm0eaoC7YHYF5FtEl4QOQAs0cuJA6u5rH63wpvaSiE4sbtdPGfN_d2g&usqp=CAE"},
 {'title':"Boys Regatta Blue  Splash Wellies - Blue", 'image_url':"https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcSEo19vaztPuOG3dlBuBomzhiDGKGSXyvOCCYaza8MARUddv6A7b9hJHIWausXH7TLyK6GD7-0NahWHsa1y5YkEv0NiSsisSSlBuNl-UZHGaScBrDrdlGrpUw&usqp=CAE"},
 {'title':"Colorful Splash Flip-Flops for Girls - Vibrant Summer Footwear Multi / UK1 / 7.5-8 Years /", 'image_url':"https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcQPwHuvGJTj4l1PHOfIGDxipgrBl6ElySomnhEAVWzEMjfwsrP5W97AIQDkGLkNOVf7QGH3CIWSh5cXTn8RjaSOGFjbve3qISPH3L-SrtYc&usqp=CAE"},
 {'title':'CerdÃ Lifes Little Moments Unisex Kids Peppa Pig Flip Flops', 'image_url':"https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcTiUgVOIFsqIwxhgl_-TvaNatuKtNrC3fjw5FPQtLGTWhfLty_jxQ0eo_osJL7ENf2pdk4kwzpFg8g029_8ERrgjCnXmBbBSyrFDZvHXX3NU4NJzY62tqU7&usqp=CAE"},
 {'title':'Nike Court Vision UK 7', 'image_url':"https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcQTSm4u-vQDmMPbJuyCPC7Cg_yvreKaWyacDdhLPgm3j8aZtRBj7gNIg-ageiEvbruaQy5AnneD9e-T3nx8Hp242HiOra6fHWGhFXltJzr5&usqp=CAE"},
 {'title':'Beautiful Hail Durga Maa Wall Art | Terracotta Wall Hanging For Decor', 'image_url':"https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcQvhyEelAf7Acqu6-HdUaPt1n9rJyQSbA4bJBJX1665RQ86X9zPa2Ol5y9nLxwZbf1Hqd8QuAzwf4_RLAdrnvM7CcS8JID96w&usqp=CAY"},
 {'title':'Rainbow Feathers Terracotta Handpainted Maa Durga Family Wall Hanging in Copper Color | Ekchala Durg', 'image_url':"https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcSYRnhz9guyTbQf7gffAxC16Laytq1tyjwkReKdFVGHvholRaUkqBtOswAKpwxpOE1uF7hR11Od8bTyZWiocw9F3Kx7op3iecyeZ4ohMb9zerygUaiSqo43&usqp=CAE"},
 {'title':'Yellow Bee Peppa Pig Pool Party Flip-Flops for Girls, Multi, UK1 / 7.5-8 Years / 20cm', 'image_url':"https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcRDBYffJwg_oC2f0QkjtRBEKJnp8z0ZXvq3rbROuEV-2o6DlHrLSjT39eviI5Qx79a3O8vkxkhNVO8igjStbD-OWypFGpCDJSKIlbpr548&usqp=CAE"},
 {'title':'Elivor Folding Spinning With Light Digital Watch', 'image_url':'https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcQLZL8pz2VGJfuOaxv0xxoz8lR6IeXRbeixF2pvWHb3ryeb8YW0zxFHHmODFXULEP5DJSpf-Wgc4Fzg3SaWzGr4jS71k2UYg-zAyxqEj5GIik2il6CrwORD&usqp=CAE'},
 {'title':'Ji And Ja Dino Dino Analog Watch - For Boys & GirlsOpens in a new window', 'image_url':"https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcSrThql1y7eYhcb7szzedBmYKAYhLCHWjB6FyAu3AcwBpbVR1fVsb8FYp1SWY1JsHjY7MEwqZPjXVdyW3QT2imqB6d_b-P73kSE6a6HqrQ4yh_cR_C1CLSM2w&usqp=CAE"},
 {'title':'ZOOP Boys Dial Analog Watch', 'image_url':"https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcS8aKPBaZkk-JsL1N2qVsAHrXLXFGLEXE-lWHypvtESWjRcJMB383By4m9LGhCRV6-ou43-tQK_0ovZGI4Ld6oE9-zWl4KVICUEeA_0wtl_nf9rrS9StmTw&usqp=CAE"},
 {'title':'Rozhok 3d Cartoon Design Strap With 7 Color Dancing Light Features Watch For Kids Analog Watch - For Boys & Girls', 'image_url':"https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcRS3AjdvNJA-eQrNCPFl8Otu5gkrAY6LaFQwY6TSBo1uT_BGRC8PDs2Ai0JHorGQB8lcehfjidZ5Odj4xIBBn3m-j9weUfABpi-srjQVys_&usqp=CAE"},
 {'title':'Time Up Baby Astro Cartoon Dials,Calling,Speaker,Fitness,Music 7-16 Years School Kids Smartwatch', 'image_url':"https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcTPdYstG_Hly2ZqHZGQdNHB4CTmxFElEAhcM5iN4LcfGTLZiET3-QYM2itc8WvBNLITGZWFSlS_E0Ul821ct7KP-OOc0dq1QivT80bt_3A&usqp=CAE"},
 {'title':'Atomic T&j_Y Atomic Series Analog Watch - For Boys & Girls', 'image_url':'https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcRCRRFsTmlTXWuTJr9rD5zZLx4GvbDkeArF-hRPmFTJIYrkBOA_7RWtryNdGD99bVjZg5LEpgGP7NPe9iEyX4dNHySGRJKGYa2Xh4jLcMOP1ccozd5UaFq2&usqp=CAE'},
 {'title':'PTJTRL Mens Classic Shawl Collar Suit Jacket Single Button Wedding Business Men Blazer Designs Hombre Masculino Slim Fit', 'image_url':'https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcSlpqLldFARWEDtPUD9RLPhUHE3P8yK-L-TpoLxJF659kK5dX-gMkZmW3qWo25vn2Yb4xdcc_5tqMK6cf95ZufKKAPJiOhNu4qtNwahe7Er&usqp=CAE'},
 {'title':'2024 Fashion New Men Leisure Boutique Business Solid Color Wedding Suit Coat Pants Vest 3 Pcs Set Dress Blazers Jacket Trousers Khaki / 3XL Is Eur XL', 'image_url':'https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcTKh5L6XRNtVKRT1dE3YU_WdDwSYWrFO71FeIA_3Ls0n4BQ8yTKtpd8cvCNnuXwRp86pZq8WSUNg9s5sLLJRrdLzQG4upHEXn6wcEYVYYs&usqp=CAE'},
 {'title':'boAt Rockerz 450 Edition Bluetooth Wireless On Ear Headphones with mic, 15 Hours Battery, 40mm Drivers, Padded Ear Cushions, Easy Access', 'image_url':"https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcSmeUMVsa9YjbNCWCTcpZZgL_ykykqZ21thnzcIwLZAD-NboumoTD8jYNkJICMuSALmm9mkEsbHRfn9ySz_HzZVmGtUEdcSisDHuaBow5K8bxeOJ4oHe44bBg&usqp=CAE"},
 {'title':'macmerise Unisex P47 Bluetooth Wireless On-Ear Headphones With Mic(Onesize) ', 'image_url':'https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcTVUUa9z1LGcJT4O1QmcXDanr5C3HLttuKOrbdQnDzIB_lUIovPaZV-o_xj1SQkOBML4M_29n2l5LsA-kuen7u6vnwxP_XA5KiHMPdntU5W47s0JZEEU6fKYg&usqp=CAE'},
 {'title':'Avengers Wireless Bluetooth Headset - Blue', 'image_url':"https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcRRJFPVzQLoPC6nAeCXjrLYoVUFhw6ktpyt4ixFGIMs80BE-btaR8beA-jigP04UdOCEnHcM7MHjSvwYiQ23WKMWibCnWuT3txtC9Gv6zniRKgCuB41JAQIuw&usqp=CAE"},
 {'title':'Marvel Wireless Bluetooth Headphones', 'image_url':'https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcRI98yhHI9GKIpYInY7v-IAOe3vU7nqVJQU_D1qkYSPaRtLVUJ3nWlnCemX7oAYyaZ6ld2bgzayG4sXf6EwrmWkVfckNdR1Z1isAbvXEgG72te2Cky9lVgkiQ&usqp=CAE'},
 {'title':'boAt Airdopes 131', 'image_url':"https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcSX0mXlZ29fAUkdcw32kqTF03qiUJmdUWvHhLOAjZ-ttCq36oMLXqhrMFdTXzJ4oQTZ5PiN-O-9UvXUhVDPnANloUrB9RowpryxVeT0AGDWNuXrsesOBqK48Q&usqp=CAE"},
{'title':'Wireless Headphone by Reconnect, OVM DBTH501 IM', 'image_url':"https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcQsIqCP3AxdFOVSHKxq-XCN19dyg3llAhL2xWjkWIM3la1ohmsl4TxRoBWxzLnqTGgGQ_g_6XrPxVEDsFbbHNO7ap033d7T-xxBdR8IbBJOyWuuYPAaKb0L&usqp=CAE"},
{'title': '2019 brand casual jacket terno masculino Mens Slim Fit blazer Black Coat', 'image_url': "https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcQ9y6SUVBVlhCAIlSZ17fpz1Uku32aDZyXYlQcjsJ5OUr7IUrEl-PR3asw6faDVoB_mwlwAtMPi44RHkYDJosbWSouy7jvYkro4olIzJGk&usqp=CAE"}
]   